In [2]:
import numpy as np

In [3]:
def rungekutta4(f, x0, t):
    num_of_nodes = np.shape(t)[0]
    x = np.zeros((num_of_nodes, np.shape(x0)[0]))
    x[0] = x0
    print("num_of_nodes = ", num_of_nodes)
    print("h = ", t[1] - t[0])
    for i in range(num_of_nodes - 1):

        h = t[i+1] - t[i]
        if i % 100000 == 0:
            print("i, x[i]: ", i, x[i])
        k1 = f(x[i], t[i])
        k2 = f(x[i] + k1 * h / 2., t[i] + h / 2.)
        k3 = f(x[i] + k2 * h / 2., t[i] + h / 2.)
        k4 = f(x[i] + k3 * h, t[i] + h)
        x[i+1] = x[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
#         print(i, h)
#         print(k1, k2, k3, k4)
    return x


In [4]:
def calc_tau(f, alpha):
    if np.max(np.abs(f)) == 0:
        return 1e-16
    else:
#         i_max = np.argmax(np.abs(f))
#         return 1e-2 * np.abs(alpha[i_max] / f[i_max])
        
#         return 1e-2 * np.min(np.abs(alpha)) / np.abs(np.max(f))
        return 1e-9 * np.linalg.norm(alpha) / np.max(np.abs(f))

def rungekutta4_t(f, x0, t_max):
    x = np.zeros((1, np.shape(x0)[0]))
    x[0] = x0
    t = 0
    while t < t_max:
        x_curr = x[-1]
        
        k1 = f(x_curr, t)
        h = calc_tau(k1, x_curr)
        print(t, h)
        k2 = f(x_curr + k1 * h / 2., t + h / 2.)
        k3 = f(x_curr + k2 * h / 2., t + h / 2.)
        k4 = f(x_curr + k3 * h, t + h)
        x_next = x_curr + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)

        x_next = x_next.reshape((1, 4))
        x = np.concatenate((x, x_next), axis=0)
        t = t+h

    return x

In [5]:
def calc_n_t(t, R_0=100., u=1e6, n_0=1e17):
    n = n_0 * (R_0 / (R_0 + u*t))**3
#     print(n)
    return n

In [6]:
def calc_T_t(t, R_0=100., u=1e6, T_0=2.):
    T = T_0 * (R_0 / (R_0 + u*t))**2
#     print(T)
    return T

In [7]:
def calc_k_1(t, A=6.06e21, I_1=5.98):
    T = calc_T_t(t)
    n = calc_n_t(t)
    k1 = A / n * T **(3./2) * np.exp(-I_1 / T)
    return k1

def calc_k_2(t, A=6.06e21, I_2=18.83):
    T = calc_T_t(t)
    n = calc_n_t(t)
    k2 = A / n * T **(3./2) * np.exp(-I_2 / T)
    return k2
    
def calc_k_3(t, A=6.06e21, I_3=28.4):
    T = calc_T_t(t)
    n = calc_n_t(t)
    k3 = A / n * T **(3./2) * np.exp(-I_3 / T)
    return k3

In [8]:
def calc_j_1(t):
    T = calc_T_t(t)
    n = calc_n_t(t)
#     print("T^4.5", pow(T, 4.5))
    return 8.75e-27 * n**2/ (T **(9./2))

def calc_j_2(t):
    T = calc_T_t(t)
    n = calc_n_t(t)
    return 7.0e-26 * n**2/ (T **(9./2))
    
def calc_j_3(t):
    T = calc_T_t(t)
    n = calc_n_t(t)
    return 2.36e-25 * n**2/ (T **(9./2))

In [9]:
def calc_func(alpha_vector, t):
    
    func = np.zeros_like(alpha_vector)
    
    alpha_0 = alpha_vector[0]
    alpha_1 = alpha_vector[1]
    alpha_2 = alpha_vector[2]
    alpha_3 = alpha_vector[3]
    
    alpha = alpha_1 + 2 * alpha_2 + 3 * alpha_3
    
    j_1 = calc_j_1(t)
    j_2 = calc_j_2(t)
    j_3 = calc_j_3(t)
    
    k_1 = calc_k_1(t)
    k_2 = calc_k_2(t)
    k_3 = calc_k_3(t)
    
#     print(t)
#     print("j:\n", j_1, j_2, j_3)
#     print("k:\n", k_1, k_2, k_3)
#     print("alpha", alpha)
#     print("diff", alpha_0 * k_1 - alpha * alpha_1)
    func[0] = - alpha * j_1 * (alpha_0 * k_1 - alpha * alpha_1)
    func[1] = alpha * j_1 * (alpha_0 * k_1 - alpha * alpha_1) - alpha * j_2 * (alpha_1 * k_2 - alpha * alpha_2)
    func[2] = alpha * j_2 * (alpha_1 * k_2 - alpha * alpha_2) - alpha * j_3 * (alpha_2 * k_3 - alpha * alpha_3)
    func[3] = alpha * j_3 * (alpha_2 * k_3 - alpha * alpha_3)

#     func[0] = - 1 * j_1 * (alpha_0 * k_1 - alpha * alpha_1)
#     func[1] = 1 * j_1 * (alpha_0 * k_1 - alpha * alpha_1) - 1 * j_2 * (alpha_1 * k_2 - alpha * alpha_2)
#     func[2] = 1 * j_2 * (alpha_1 * k_2 - alpha * alpha_2) - 1 * j_3 * (alpha_2 * k_3 - alpha * alpha_3)
#     func[3] = 1 * j_3 * (alpha_2 * k_3 - alpha * alpha_3)

#     print("func: \n", func)
#     print(np.sum(alpha_vector))
    assert func[0] == func[0]
    return func

In [10]:
# t = np.linspace(0, 1e-9, 101)
# t = np.linspace(0, 1e-3, 200000001)
t = np.linspace(0, 4 * 1e-4, 20000001)
# alpha_0 = np.array([8621.424084890414, -8634.437408398891, 14.01351351168498, -0.00019000320715256778])
alpha_0 = np.array([2.59333106e-05, 0.11553579, 0.83412872, 0.05030956])
# alpha_0 = np.array([])

# sol = rungekutta4_t(calc_func, alpha_0, 1e-2)
sol = rungekutta4(calc_func, alpha_0, t)

num_of_nodes =  20000001
h =  2.0000000000000002e-11
i, x[i]:  0 [2.59333106e-05 1.15535790e-01 8.34128720e-01 5.03095600e-02]
i, x[i]:  100000 [3.89159061e-05 1.59007551e-01 8.12361808e-01 2.85917290e-02]
i, x[i]:  200000 [5.71080808e-05 2.14085727e-01 7.70184953e-01 1.56722155e-02]
i, x[i]:  300000 [8.15114156e-05 2.81125741e-01 7.10485877e-01 8.30687431e-03]
i, x[i]:  400000 [1.12680155e-04 3.58979470e-01 6.36651944e-01 4.25590865e-03]
i, x[i]:  500000 [1.50544563e-04 4.44794802e-01 5.52951683e-01 2.10297405e-03]
i, x[i]:  600000 [1.94489400e-04 5.34329205e-01 4.64477616e-01 9.98692566e-04]
i, x[i]:  700000 [2.43697465e-04 6.22674401e-01 3.76628071e-01 4.53834071e-04]
i, x[i]:  800000 [2.97617008e-04 7.05127822e-01 2.94378125e-01 1.96438751e-04]
i, x[i]:  900000 [3.56355166e-04 7.77943623e-01 2.21619369e-01 8.06554330e-05]
i, x[i]:  1000000 [4.20864714e-04 8.38810553e-01 1.60737257e-01 3.13294580e-05]
i, x[i]:  1100000 [4.92909285e-04 8.87006575e-01 1.12489010e-01 1.15089287e-05]
i,

i, x[i]:  10200000 [9.89060578e-01 1.09394252e-02 3.69024731e-26 1.26735875e-68]
i, x[i]:  10300000 [9.89336683e-01 1.06633200e-02 3.00698235e-26 6.35472334e-69]
i, x[i]:  10400000 [9.89597856e-01 1.04021474e-02 2.46546513e-26 3.25365547e-69]
i, x[i]:  10500000 [9.89845399e-01 1.01546043e-02 2.03312876e-26 1.69851127e-69]
i, x[i]:  10600000 [9.90080464e-01 9.91953917e-03 1.68560117e-26 9.02818039e-70]
i, x[i]:  10700000 [9.90304072e-01 9.69593096e-03 1.40446942e-26 4.88022304e-70]
i, x[i]:  10800000 [9.90517132e-01 9.48287123e-03 1.17569944e-26 2.67985639e-70]
i, x[i]:  10900000 [9.90720454e-01 9.27954914e-03 9.88506284e-27 1.49343408e-70]
i, x[i]:  11000000 [9.90914765e-01 9.08523866e-03 8.34537874e-27 8.43866036e-71]
i, x[i]:  11100000 [9.91100716e-01 8.89928772e-03 7.07279371e-27 4.83080275e-71]
i, x[i]:  11200000 [9.91278894e-01 8.72110894e-03 6.01614530e-27 2.79962590e-71]
i, x[i]:  11300000 [9.91449832e-01 8.55017170e-03 5.13499803e-27 1.64142018e-71]
i, x[i]:  11400000 [9.916140

In [12]:
import pickle

In [13]:
# with open('data.pickle', 'wb') as f:
#     pickle.dump(sol, f)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot()

plt.plot(t, sol[:,0], label="alpha0")
plt.plot(t, sol[:,1], label="alpha1")
plt.plot(t, sol[:,2], label="alpha2")
plt.plot(t, sol[:,3], label="alpha3")
ax.legend()
ax.grid()
ax.set(xlabel='time')

plt.savefig("res.png")